# 모든 종목 재무제표 크롤링 

In [ ]:
# 전체 재무제표 크롤링한다음에 여기서 코드 뽑아쓰면 되는데,,ㅅㅂ 
# 전체 재무제표에서 조건에 맞는 종목들 추출 (분기별)
# 반복문으로 20일전 수익률, 60일 전후 수익률 만들고 20일전 수익률이 5%이상인 종목 추출 
#반복문으로 그 종목의 분기를 합침 
# 평균 수익률 계산 

In [1]:
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

In [2]:
marcapdf = pd.read_csv('./data/marcap/marcap/data/marcap-2022.csv.gz')

In [3]:
marcapdf

,Code,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId,Rank,Date
0,005930,삼성전자,KOSPI,NaN,78600.0,1,300.0,0.38,79400.0,79800.0,78200.0,13502112.0,1.066007e+12,4.692249e+14,5969782550,STK,1,2022-01-03
1,000660,SK하이닉스,KOSPI,NaN,128500.0,2,-2500.0,-1.91,132000.0,132500.0,128000.0,3782213.0,4.909400e+11,9.354830e+13,728002365,STK,2,2022-01-03
2,035420,NAVER,KOSPI,NaN,376000.0,2,-2500.0,-0.66,379000.0,381000.0,375500.0,378920.0,1.429541e+11,6.168246e+13,164049085,STK,3,2022-01-03
3,207940,삼성바이오로직스,KOSPI,NaN,911000.0,1,8000.0,0.89,903000.0,920000.0,902000.0,46043.0,4.191969e+10,6.027632e+13,66165000,STK,4,2022-01-03
4,005935,삼성전자우,KOSPI,NaN,71400.0,1,200.0,0.28,71800.0,71800.0,71300.0,832097.0,5.953141e+10,5.875411e+13,822886700,STK,5,2022-01-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225121,009275,신원우,KOSPI,NaN,32000.0,2,-450.0,-1.39,31250.0,32450.0,31200.0,326.0,1.024625e+07,2.903680e+09,90740,STK,2619,2022-05-10
225122,215050,비엔디생활건강,KONEX,일반기업부,912.0,0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000e+00,1.934959e+09,2121666,KNX,2620,2022-05-10
225123,329020,오션스톤,KONEX,일반기업부,1700.0,2,-250.0,-12.82,2000.0,2200.0,1660.0,3576.0,6.354905e+06,1.419758e+09,835152,KNX,2621,2022-05-10
225124,267060,명진홀딩스,KONEX,일반기업부,154.0,5,-27.0,-14.92,192.0,192.0,154.0,2158.0,3.327030e+05,1.407838e+09,9141804,KNX,2622,2022-05-10


In [4]:
marcapdf['Codes'] = marcapdf['Code'].drop_duplicates()

In [5]:
marcapdf = marcapdf.drop('Dept',axis=1)
marcapdf

,Code,Name,Market,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId,Rank,Date,Codes
0,005930,삼성전자,KOSPI,78600.0,1,300.0,0.38,79400.0,79800.0,78200.0,13502112.0,1.066007e+12,4.692249e+14,5969782550,STK,1,2022-01-03,005930
1,000660,SK하이닉스,KOSPI,128500.0,2,-2500.0,-1.91,132000.0,132500.0,128000.0,3782213.0,4.909400e+11,9.354830e+13,728002365,STK,2,2022-01-03,000660
2,035420,NAVER,KOSPI,376000.0,2,-2500.0,-0.66,379000.0,381000.0,375500.0,378920.0,1.429541e+11,6.168246e+13,164049085,STK,3,2022-01-03,035420
3,207940,삼성바이오로직스,KOSPI,911000.0,1,8000.0,0.89,903000.0,920000.0,902000.0,46043.0,4.191969e+10,6.027632e+13,66165000,STK,4,2022-01-03,207940
4,005935,삼성전자우,KOSPI,71400.0,1,200.0,0.28,71800.0,71800.0,71300.0,832097.0,5.953141e+10,5.875411e+13,822886700,STK,5,2022-01-03,005935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225121,009275,신원우,KOSPI,32000.0,2,-450.0,-1.39,31250.0,32450.0,31200.0,326.0,1.024625e+07,2.903680e+09,90740,STK,2619,2022-05-10,NaN
225122,215050,비엔디생활건강,KONEX,912.0,0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000e+00,1.934959e+09,2121666,KNX,2620,2022-05-10,NaN
225123,329020,오션스톤,KONEX,1700.0,2,-250.0,-12.82,2000.0,2200.0,1660.0,3576.0,6.354905e+06,1.419758e+09,835152,KNX,2621,2022-05-10,NaN
225124,267060,명진홀딩스,KONEX,154.0,5,-27.0,-14.92,192.0,192.0,154.0,2158.0,3.327030e+05,1.407838e+09,9141804,KNX,2622,2022-05-10,NaN


In [6]:
marcapdf.dropna(inplace=True)
marcapdf

,Code,Name,Market,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId,Rank,Date,Codes
0,005930,삼성전자,KOSPI,78600.0,1,300.0,0.38,79400.0,79800.0,78200.0,13502112.0,1.066007e+12,4.692249e+14,5969782550,STK,1,2022-01-03,005930
1,000660,SK하이닉스,KOSPI,128500.0,2,-2500.0,-1.91,132000.0,132500.0,128000.0,3782213.0,4.909400e+11,9.354830e+13,728002365,STK,2,2022-01-03,000660
2,035420,NAVER,KOSPI,376000.0,2,-2500.0,-0.66,379000.0,381000.0,375500.0,378920.0,1.429541e+11,6.168246e+13,164049085,STK,3,2022-01-03,035420
3,207940,삼성바이오로직스,KOSPI,911000.0,1,8000.0,0.89,903000.0,920000.0,902000.0,46043.0,4.191969e+10,6.027632e+13,66165000,STK,4,2022-01-03,207940
4,005935,삼성전자우,KOSPI,71400.0,1,200.0,0.28,71800.0,71800.0,71300.0,832097.0,5.953141e+10,5.875411e+13,822886700,STK,5,2022-01-03,005935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204046,405640,신한제9호스팩,KOSDAQ,2255.0,1,15.0,0.67,2240.0,2270.0,2240.0,1724965.0,3.878725e+09,1.022642e+10,4535000,KSQ,2530,2022-04-27,405640
204658,389140,포바이포,KOSDAQ,44200.0,4,10200.0,30.00,34000.0,44200.0,34000.0,516447.0,2.267102e+10,4.519112e+11,10224235,KSQ,522,2022-04-28,389140
205930,37550L,DL이앤씨2우(전환),KOSPI,43450.0,1,1850.0,4.45,41600.0,45500.0,41000.0,184729.0,7.963149e+09,9.176427e+10,2111951,STK,1794,2022-04-28,37550L
206676,419270,신영스팩7호,KOSDAQ,2270.0,2,-25.0,-1.09,2295.0,2310.0,2270.0,1132665.0,2.593768e+09,9.772350e+09,4305000,KSQ,2540,2022-04-28,419270


In [7]:
codes = marcapdf['Codes'].tolist()

In [8]:
codes

['005930',
 '000660',
 '035420',
 '207940',
 '005935',
 '035720',
 '005380',
 '006400',
 '051910',
 '000270',
 '323410',
 '068270',
 '005490',
 '012330',
 '377300',
 '105560',
 '096770',
 '066570',
 '259960',
 '028260',
 '055550',
 '034730',
 '302440',
 '051900',
 '036570',
 '009150',
 '352820',
 '015760',
 '011200',
 '032830',
 '003550',
 '086790',
 '017670',
 '091990',
 '018260',
 '361610',
 '003670',
 '251270',
 '034020',
 '033780',
 '247540',
 '003490',
 '090430',
 '000810',
 '010950',
 '010130',
 '402340',
 '316140',
 '263750',
 '034220',
 '011070',
 '329180',
 '030200',
 '326030',
 '024110',
 '011170',
 '066970',
 '383220',
 '293490',
 '018880',
 '009830',
 '009540',
 '086280',
 '011790',
 '112040',
 '138040',
 '020150',
 '032640',
 '097950',
 '088980',
 '137310',
 '004020',
 '006800',
 '021240',
 '035250',
 '000720',
 '011780',
 '161390',
 '010140',
 '068760',
 '028050',
 '071050',
 '000100',
 '267250',
 '139480',
 '000060',
 '180640',
 '271560',
 '241560',
 '016360',
 '003410',

In [9]:
# 2022년도 재무제표 
import requests
from bs4 import BeautifulSoup

from selenium.webdriver.common.keys import Keys
import time
from selenium import webdriver

driver  =webdriver.Chrome()
driver.implicitly_wait(1)
count = 0

for code in codes[2200:]:

    url = 'https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cn=&cmp_cd={}&menuType=block'.\
            format(code)
    try:
        driver.get(url)
#         element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "cns_Tab22")))        
        try:
            driver.find_element_by_id('cns_Tab22').click()

            html = driver.page_source

            tables = pd.read_html(html, encoding='utf-8')
            fin_df = tables[12]

            fin_df = fin_df.T.iloc[:6]
            fin_df.columns = fin_df.iloc[0]
            fin_df.drop('주요재무정보', inplace=True)

            fin_df.reset_index(inplace=True)
            fin_df.drop('level_0', axis=1, inplace=True)

            fin_df.index = fin_df.level_1.apply(lambda x: x.split()[0])
            fin_df.drop('level_1', axis=1, inplace=True)

            fin_df = fin_df[['매출액','영업이익','당기순이익','자산총계','부채총계','자본총계','ROE(%)', 'ROA(%)',\
                    'EPS(원)', 'PER(배)','BPS(원)', 'PBR(배)']]

            fin_df['code'] = code

            fin_df.to_csv('./data/네이버재무제표_{}'.format(code))

            count +=1
            print(count, end=' ')
        except:
            pass
    except:
        pass

C:\Users\bitcamp\AppData\Local\Temp/ipykernel_8980/2927220948.py:21: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id('cns_Tab22').click()
C:\Users\bitcamp\anaconda3\lib\site-packages\pandas\core\generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

In [ ]:
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

# driver = webdriver.Chrome('chromedriver.exe')
# driver.get("https://pythondocs.net")

# element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "idname")))

In [ ]:
# 전체 재무제표 크롤링한다음에 여기서 코드 뽑아쓰면 되는데,,ㅅㅂ 
# 전체 재무제표에서 조건에 맞는 종목들 추출 (분기별)
# 반복문으로 20일전 수익률, 60일 전후 수익률 만들고 20일전 수익률이 5%이상인 종목 추출 
#반복문으로 그 종목의 분기를 합침 
# 평균 수익률 계산 

In [ ]:
import os 
code_lis = os.listdir('./data/네이버재무제표')